In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_astradb import AstraDBVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
import pandas as pd
import os

c:\a\envs\ayurveda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-pro",temperature = 0.8)

print(llm.invoke("What is capital of India").content)

The capital of India is **New Delhi**. 



In [3]:
symptoms_data = pd.read_csv(r"C:\Users\aashutosh kumar\projects\Ayurvedic-Remedy-Bot\data\ayurvedic_symptoms_desc.csv")
symptoms_data.head(4)

,Symptom,Description
0,Vatavikara,Disorders related to the Vata dosha.
1,Netraroga,Eye disorders.
2,Malavarodha,Constipation.
3,Sutikadosha,Postpartum disorders.


In [4]:
medication_data = pd.read_csv(r"C:\Users\aashutosh kumar\projects\Ayurvedic-Remedy-Bot\data\Formulation-Indications.csv")
medication_data.head(4)

,Name of Medicine,Reference text,Dispensing Pack Size,Main Indications,Dose,Precaution/ Contraindication,Preferred use (OPD/ IPD),Class
0,Abhayarishta,AFI,200 ml,"Arsha, Agnimandya,\nUdararoga, Vibandha",12 - 24 ml,NS,Both,A
1,Amritarishta,AFI,200 ml,"SarvaJvara, Jirna Jvara",12 - 24 ml,NS,Both,A
2,Aragvadharishta,AH,200ml,"Kandu, Tvak Vikara,\nVibandha",12 - 24 ml,NS,Both,A
3,Aravindasava,AFI,200 ml,"Balaroga, Balakshaya,\nAgnimandya, Aruchi",12 - 24 ml,NS,Both,A


In [5]:
class_labels = pd.read_csv(r"data\FormulationClass.csv")
class_labels.head(3)

,Class,Unnamed: 1
0,A,Asava Arista
1,B,Arka Kalpana
2,C,"Avaleha, Khand, Pak Kalpana"


In [6]:
classes = dict(zip(class_labels["Class"],class_labels["Unnamed: 1"]))
medication_data["Class"] = medication_data["Class"].map(classes)

In [7]:
medication_data.head()

,Name of Medicine,Reference text,Dispensing Pack Size,Main Indications,Dose,Precaution/ Contraindication,Preferred use (OPD/ IPD),Class
0,Abhayarishta,AFI,200 ml,"Arsha, Agnimandya,\nUdararoga, Vibandha",12 - 24 ml,NS,Both,Asava Arista
1,Amritarishta,AFI,200 ml,"SarvaJvara, Jirna Jvara",12 - 24 ml,NS,Both,Asava Arista
2,Aragvadharishta,AH,200ml,"Kandu, Tvak Vikara,\nVibandha",12 - 24 ml,NS,Both,Asava Arista
3,Aravindasava,AFI,200 ml,"Balaroga, Balakshaya,\nAgnimandya, Aruchi",12 - 24 ml,NS,Both,Asava Arista
4,Arjunarishta/ Parthadyarishta,AFI,200 ml,"Hridroga, Hriddrava, Hrid- daurbalya, Moha,\nM...",12 - 24 ml,NS,Both,Asava Arista


In [8]:
def csv_to_document(data):
    list = []

    for i,row in data.iterrows():
        dict = {
            "product_name" : row["product_title"],
            "review" : row["review"]
        }
        product_list.append(dict)

In [9]:
medication_data.head(2)

,Name of Medicine,Reference text,Dispensing Pack Size,Main Indications,Dose,Precaution/ Contraindication,Preferred use (OPD/ IPD),Class
0,Abhayarishta,AFI,200 ml,"Arsha, Agnimandya,\nUdararoga, Vibandha",12 - 24 ml,NS,Both,Asava Arista
1,Amritarishta,AFI,200 ml,"SarvaJvara, Jirna Jvara",12 - 24 ml,NS,Both,Asava Arista


In [39]:
medication_data.isnull().sum()
medication_data.dropna(inplace=True)

**PREPARING THE DATASET OF SYMPTOMS AND DESC**

In [40]:
symptom_list = []
for i,row in symptoms_data.iterrows():
    dict = {
        "Symptom" : row["Symptom"],
        "Description" : row["Description"] 
    }
    symptom_list.append(dict)

symptom_list[:6]

[{'Symptom': 'Vatavikara',
  'Description': 'Disorders related to the Vata dosha.'},
 {'Symptom': 'Netraroga', 'Description': 'Eye disorders.'},
 {'Symptom': 'Malavarodha', 'Description': 'Constipation.'},
 {'Symptom': 'Sutikadosha', 'Description': 'Postpartum disorders.'},
 {'Symptom': 'Vrana', 'Description': 'Wounds or injuries.'},
 {'Symptom': 'Khalitya', 'Description': 'Hair loss.'}]

In [11]:
symp_docs = []
for dict in symptom_list:
    metadata = {"Symptom" : dict["Symptom"]}
    doc = Document(page_content = dict["Description"],metadata = metadata)
    symp_docs.append(doc)
symp_docs[:5]

[Document(metadata={'Symptom': 'Vatavikara'}, page_content='Disorders related to the Vata dosha.'),
 Document(metadata={'Symptom': 'Netraroga'}, page_content='Eye disorders.'),
 Document(metadata={'Symptom': 'Malavarodha'}, page_content='Constipation.'),
 Document(metadata={'Symptom': 'Sutikadosha'}, page_content='Postpartum disorders.'),
 Document(metadata={'Symptom': 'Vrana'}, page_content='Wounds or injuries.')]

**CONVERTING MEDICATIONS DATFRAME TO DOCUMENT**

In [41]:
medicines_list = []

for i, row in medication_data.iterrows():
    med_dict = {
        "Name of Medicine": row["Name of Medicine"],
        "Reference text": row["Reference text"],
        "Dispensing Pack Size": row["Dispensing Pack Size"],
        "Main Indications": row["Main Indications"],
        "Dose": row["Dose"],
        "Precaution/ Contraindication": row["Precaution/ Contraindication"],
        "Preferred use (OPD/ IPD)": row["Preferred use (OPD/ IPD)"],
        "Class": row["Class"]
    }
    medicines_list.append(med_dict)

medicines_list[:2]

[{'Name of Medicine': 'Abhayarishta',
  'Reference text': 'AFI',
  'Dispensing Pack Size': '200 ml',
  'Main Indications': 'Arsha, Agnimandya,\nUdararoga, Vibandha',
  'Dose': '12 - 24 ml',
  'Precaution/ Contraindication': 'NS',
  'Preferred use (OPD/ IPD)': 'Both',
  'Class': 'Asava Arista '},
 {'Name of Medicine': 'Amritarishta',
  'Reference text': 'AFI',
  'Dispensing Pack Size': '200 ml',
  'Main Indications': 'SarvaJvara, Jirna Jvara',
  'Dose': '12 - 24 ml',
  'Precaution/ Contraindication': 'NS',
  'Preferred use (OPD/ IPD)': 'Both',
  'Class': 'Asava Arista '}]

In [36]:
med_docs = []

for med_dict in medicines_list:
    metadata = {
        "Name_of_Medicine": med_dict["Name of Medicine"],
        "Reference_text": med_dict["Reference text"],
        "Dispensing_Pack_Size": med_dict["Dispensing Pack Size"],
        "Main_Indications": med_dict["Main Indications"],
        "Dose": med_dict["Dose"],
        "Precaution/Contraindication": med_dict["Precaution/ Contraindication"],
        "Preferred_use": med_dict["Preferred use (OPD/ IPD)"],
        "Class": med_dict["Class"]
    }
    doc = Document(page_content=f"Name of Medicine: {med_dict['Name of Medicine']}\n"
                                f"Reference text: {med_dict['Reference text']}\n"
                                f"Dispensing Pack Size: {med_dict['Dispensing Pack Size']}\n"
                                f"Main Indications: {med_dict['Main Indications']}\n"
                                f"Dose: {med_dict['Dose']}\n"
                                f"Precaution/ Contraindication: {med_dict['Precaution/ Contraindication']}\n"
                                f"Preferred use (OPD/ IPD): {med_dict['Preferred use (OPD/ IPD)']}\n"
                                f"Class: {med_dict['Class']}",
                      metadata=metadata)
    med_docs.append(doc)

med_docs[:5]


[Document(metadata={'Name_of_Medicine': 'Abhayarishta', 'Reference_text': 'AFI', 'Dispensing_Pack_Size': '200 ml', 'Main_Indications': 'Arsha, Agnimandya,\nUdararoga, Vibandha', 'Dose': '12 - 24 ml', 'Precaution': 'NS', 'Preferred_use': 'Both', 'Class': 'Asava Arista '}, page_content='Name of Medicine: Abhayarishta\nReference text: AFI\nDispensing Pack Size: 200 ml\nMain Indications: Arsha, Agnimandya,\nUdararoga, Vibandha\nDose: 12 - 24 ml\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Asava Arista '),
 Document(metadata={'Name_of_Medicine': 'Amritarishta', 'Reference_text': 'AFI', 'Dispensing_Pack_Size': '200 ml', 'Main_Indications': 'SarvaJvara, Jirna Jvara', 'Dose': '12 - 24 ml', 'Precaution': 'NS', 'Preferred_use': 'Both', 'Class': 'Asava Arista '}, page_content='Name of Medicine: Amritarishta\nReference text: AFI\nDispensing Pack Size: 200 ml\nMain Indications: SarvaJvara, Jirna Jvara\nDose: 12 - 24 ml\nPrecaution/ Contraindication: NS\nPreferred use (O

**STORING THE DOCUMENTS TO VECTORSTORE**

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001B953136800>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [15]:
load_dotenv()
ASTRA_API_KEY = os.getenv("ASTRA_API")
DB_ENDPOINT = os.getenv("DB_ENDPOINT")
DB_ID = os.getenv("DB_ID")

In [17]:
vector_store = AstraDBVectorStore(collection_name = "Ayurvedic_Bot",
                               embedding = embeddings,
                               api_endpoint = DB_ENDPOINT,
                               token = ASTRA_API_KEY,
                               namespace = "symptoms")
    
# inserted_ids = vector_store.add_documents(symp_docs)
# print(f"Inserted {len(inserted_ids)} documents")



In [ ]:
inserted_ids = vector_store.add_documents(symp_docs)
print(f"Inserted {len(inserted_ids)} documents")

In [27]:
results = vector_store.similarity_search("eye disorder")
symptom_names = []

for res in results:
    symptom_name = res.metadata.get("Symptom", "Unknown")
    print(f"* {res.page_content} [{res.metadata}]")
    symptom_names.append(symptom_name)

print(symptom_names[0])

* Eye disorders. [{'Symptom': 'Netraroga'}]
* Ophthalmia. [{'Symptom': 'Abhishyanda'}]
* Blindness. [{'Symptom': 'Timira'}]
* Eye injury. [{'Symptom': 'Netravrana'}]
Netraroga


In [42]:
medication_vector_store = AstraDBVectorStore(collection_name = "Ayurvedic_Bot",
                               embedding = embeddings,
                               api_endpoint = DB_ENDPOINT,
                               token = ASTRA_API_KEY,
                               namespace = "default_keyspace")

In [44]:
def clean_metadata(doc):
    for key, value in doc.metadata.items():
        if isinstance(value, float):
            if not (value > float('-inf') and value < float('inf')):
                doc.metadata[key] = None
    return doc

med_docs = [clean_metadata(doc) for doc in med_docs]

inserted_ids1 = medication_vector_store.add_documents(med_docs)
print(f"Inserted {len(inserted_ids1)} documents")


Inserted 202 documents


In [47]:
def enhanced_similarity_search(vector_store, query, field_weights=None):
    # Use a field-specific search if the vector store supports it
    # This example assumes field_weights is a dictionary with field names and their corresponding weights
    if field_weights is None:
        field_weights = {'Main_Indications': 1.0, 'page_content': 0.5}
    
    results = vector_store.similarity_search(query, field_weights=field_weights)
    return results

# Perform the enhanced similarity search for a disease
query = "Netraroga"
result = enhanced_similarity_search(medication_vector_store, query)

for doc in result:
    print(f"Document:\n{doc.page_content}\nMetadata:\n{doc.metadata}\n")

Document:
Name of Medicine: Nalpamaradi Taila
Reference text: AFI
Dispensing Pack Size: 25 ml
Main Indications: Tvakroga, Kushta,
Pama, Kandu
Dose: For  Pratisarana,
Abhyanga
Precaution/ Contraindication: NS
Preferred use (OPD/ IPD): Both
Class: Taila 
Metadata:
{'Name_of_Medicine': 'Nalpamaradi Taila', 'Reference_text': 'AFI', 'Dispensing_Pack_Size': '25 ml', 'Main_Indications': 'Tvakroga, Kushta,\nPama, Kandu', 'Dose': 'For  Pratisarana,\nAbhyanga', 'Precaution': 'NS', 'Preferred_use': 'Both', 'Class': 'Taila '}

Document:
Name of Medicine: Nalpamaradi Taila
Reference text: AFI
Dispensing Pack Size: 25 ml
Main Indications: Tvakroga, Kushta,
Pama, Kandu
Dose: For  Pratisarana,
Abhyanga
Precaution/ Contraindication: NS
Preferred use (OPD/ IPD): Both
Class: Taila 
Metadata:
{'Name_of_Medicine': 'Nalpamaradi Taila', 'Reference_text': 'AFI', 'Dispensing_Pack_Size': '25 ml', 'Main_Indications': 'Tvakroga, Kushta,\nPama, Kandu', 'Dose': 'For  Pratisarana,\nAbhyanga', 'Precaution': 'NS', 'Pr

In [48]:
med_docs1 = []

for med_dict in medicines_list:
    metadata = {
        "Main_Indications": med_dict["Main Indications"]
    }
    doc = Document(page_content=f"Name of Medicine: {med_dict['Name of Medicine']}\n"
                                f"Reference text: {med_dict['Reference text']}\n"
                                f"Dispensing Pack Size: {med_dict['Dispensing Pack Size']}\n"
                                f"Main Indications: {med_dict['Main Indications']}\n"
                                f"Dose: {med_dict['Dose']}\n"
                                f"Precaution/ Contraindication: {med_dict['Precaution/ Contraindication']}\n"
                                f"Preferred use (OPD/ IPD): {med_dict['Preferred use (OPD/ IPD)']}\n"
                                f"Class: {med_dict['Class']}",
                      metadata=metadata)
    med_docs1.append(doc)

med_docs1[:5]


[Document(metadata={'Main_Indications': 'Arsha, Agnimandya,\nUdararoga, Vibandha'}, page_content='Name of Medicine: Abhayarishta\nReference text: AFI\nDispensing Pack Size: 200 ml\nMain Indications: Arsha, Agnimandya,\nUdararoga, Vibandha\nDose: 12 - 24 ml\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Asava Arista '),
 Document(metadata={'Main_Indications': 'SarvaJvara, Jirna Jvara'}, page_content='Name of Medicine: Amritarishta\nReference text: AFI\nDispensing Pack Size: 200 ml\nMain Indications: SarvaJvara, Jirna Jvara\nDose: 12 - 24 ml\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Asava Arista '),
 Document(metadata={'Main_Indications': 'Kandu, Tvak Vikara,\nVibandha'}, page_content='Name of Medicine: Aragvadharishta\nReference text: AH\nDispensing Pack Size: 200ml\nMain Indications: Kandu, Tvak Vikara,\nVibandha\nDose: 12 - 24 ml\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Asava Arista '),
 Docume

In [49]:
medication_vector_store = AstraDBVectorStore(collection_name = "Ayurvedic_Bot",
                               embedding = embeddings,
                               api_endpoint = DB_ENDPOINT,
                               token = ASTRA_API_KEY,
                               namespace = "medications")

In [50]:
def clean_metadata(doc):
    for key, value in doc.metadata.items():
        if isinstance(value, float):
            if not (value > float('-inf') and value < float('inf')):
                doc.metadata[key] = None
    return doc

med_docs1 = [clean_metadata(doc) for doc in med_docs1]

inserted_ids1 = medication_vector_store.add_documents(med_docs1)
print(f"Inserted {len(inserted_ids1)} documents")


Inserted 185 documents


In [52]:
result = medication_vector_store.similarity_search("Netraroga")
result

[Document(metadata={'Main_Indications': 'Tvakroga, Kushta,\nPama, Kandu'}, page_content='Name of Medicine: Nalpamaradi Taila\nReference text: AFI\nDispensing Pack Size: 25 ml\nMain Indications: Tvakroga, Kushta,\nPama, Kandu\nDose: For  Pratisarana,\nAbhyanga\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Taila '),
 Document(metadata={'Main_Indications': 'Nadivrana, Pama, Apachi, Gandamala, Galaganda'}, page_content='Name of Medicine: Nirgundi Taila\nReference text: BR\nDispensing Pack Size: 25 ml\nMain Indications: Nadivrana, Pama, Apachi, Gandamala, Galaganda\nDose: Internal  use: 15- 30 ml\nExternal use: Abhyanga and 2-6 drops for Nasya or\nKarnapuran\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Taila '),
 Document(metadata={'Main_Indications': 'Keshapata, Shiroroga, Khalitya, Indralupta'}, page_content='Name of Medicine: Bhringaraja Tail\nReference text: BR\nDispensing Pack Size: 50 ml\nMain Indications: Keshapata, Shiroroga, 